In [1]:
%cd ..

C:\Users\OLOYEDE_O\PycharmProjects\scraper


In [2]:
from iea_scraper.jobs.utils import get_driver
import pandas as pd
import selenium
from selenium import webdriver

In [3]:
url = 'https://www.petrocom.gov.gh/production-figures/'

In [4]:
# create the selenium object
driver = get_driver(headless=True)

In [5]:
# get the web content
driver.get(url)

In [6]:
import re
# get the field name and extracting the year
field_details = []
field_names = driver.find_elements_by_class_name("accordion-toggle")
for name in field_names:
    field_name = name.text.strip().split("Production",1)[0]
    temp_year = name.text.strip().split()
    production_year = re.findall('[0-9]+', str(temp_year))[0]
    field_details.append({'field_name': field_name,'year':production_year})

In [7]:
field_details

[{'field_name': 'TEN ', 'year': '2021'},
 {'field_name': 'Jubilee ', 'year': '2021'},
 {'field_name': '(OCTP) Sankofa/Gye Nyame ', 'year': '2021'}]

In [8]:
# download the link to the table in html
source_ids = []
table_links = driver.find_elements_by_tag_name('table')
for link in table_links:
    table_id = link.get_property('id')
    source_ids.append(table_id)

In [9]:
# the table unique identifier
source_ids

['tablepress-11', 'tablepress-10', 'tablepress-12']

**DATA TRANSFORMATION**

In [302]:
# read the data into pandas for transformation
df =pd.read_html(io=driver.page_source,attrs={'id':source_ids[1]})[0]

In [303]:
# retrive the content of the tables as an html
t = driver.find_element_by_id('tablepress-11')
t.get_attribute('innerHTML')

'\n<thead>\n<tr class="row-1 odd" role="row"><th class="column-1 sorting" tabindex="0" aria-controls="tablepress-11" rowspan="1" colspan="1" aria-label="TEN Production for the Year 2021: activate to sort column ascending" style="width: 0px;">TEN Production for the Year 2021</th><th class="column-2 sorting" tabindex="0" aria-controls="tablepress-11" rowspan="1" colspan="1" aria-label="&amp;nbsp;: activate to sort column ascending" style="width: 0px;">&nbsp;</th><th class="column-3 sorting" tabindex="0" aria-controls="tablepress-11" rowspan="1" colspan="1" aria-label="&amp;nbsp;: activate to sort column ascending" style="width: 0px;">&nbsp;</th><th class="column-4 sorting" tabindex="0" aria-controls="tablepress-11" rowspan="1" colspan="1" aria-label="&amp;nbsp;: activate to sort column ascending" style="width: 0px;">&nbsp;</th><th class="column-5 sorting" tabindex="0" aria-controls="tablepress-11" rowspan="1" colspan="1" aria-label="&amp;nbsp;: activate to sort column ascending" style="w

In [304]:
df.head(20)

,Jubilee Production for the Year 2021,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,Period,Oil Production (bbl),Gas Production (MMscf),Gas Re-injected (MMscf),Fuel Gas (MMscf),Flared Gas (MMscf),Gas Exported to GNGC (MMscf),Gas Export to TEN (MMscf)
1,"January, 2021",2255226.00,5728.94,1090.67,372.87,957.92,3307.48,-
2,"February, 2021",1975758.00,5187.61,1070.88,331.28,898.42,2887.03,-
3,"March, 2021",2102802.00,5502.36,1371.20,343.14,976.57,2811.45,-
4,"April, 2021",2155621.00,6044.63,1448.77,356.83,989.27,3249.76,-
5,"May, 2021",2182699.00,5986.46,1333.26,354.80,1033.51,3264.89,-
6,"June, 2021",2109221.00,5910.84,1913.76,362.32,1002.07,2632.69,-
7,"July, 2021",2379739.00,6109.60,1816.48,367.51,1044.83,2880.79,-
8,"August, 2021",2389790.00,6109.60,1633.30,349.52,1095.71,2937.83,-
9,"September, 2021",2389622.00,5957.41,2053.87,332.76,993.89,2576.89,-


In [305]:
df.columns[0]

'Jubilee Production for the Year 2021'

In [306]:
# get the field and year
important_data = df.columns[0]

In [307]:
# get the field name
field_name = important_data.strip().split("Production",1)[0]

In [308]:
field_name

'Jubilee '

In [309]:
# get the year
temp_year = important_data.strip().split()
production_year = re.findall('[0-9]+', str(temp_year))[0]

In [310]:
production_year

'2021'

In [311]:
# drop unwanted rows
df.drop([13,14],axis=0,inplace=True)

In [312]:
# drop na values
df.dropna(how='any',inplace=True)

In [313]:
# change the column name
df.columns = df.iloc[0, :]

In [314]:
col_name_gas = df.columns[2].strip()
df.rename(columns={col_name_gas: 'gas production (mmscf)'}, inplace=True)

In [315]:
production_year = df.iloc[1][0].split()[1]

In [316]:
production_year

'2021'

In [297]:
df.reset_index(drop=True,inplace=True)

In [298]:
df.head(20)

,Period,Oil Production (bbl),gas production (mmscf),Gas Re-injected (MMscf),Fuel Gas (MMscf),Flared Gas (MMscf),Gas Exported to GNGC (MMscf),Gas Export to TEN (MMscf)
0,Period,Oil Production (bbl),Gas Production (MMscf),Gas Re-injected (MMscf),Fuel Gas (MMscf),Flared Gas (MMscf),Gas Exported to GNGC (MMscf),Gas Export to TEN (MMscf)
1,"January, 2021",2255226.00,5728.94,1090.67,372.87,957.92,3307.48,-
2,"February, 2021",1975758.00,5187.61,1070.88,331.28,898.42,2887.03,-
3,"March, 2021",2102802.00,5502.36,1371.20,343.14,976.57,2811.45,-
4,"April, 2021",2155621.00,6044.63,1448.77,356.83,989.27,3249.76,-
5,"May, 2021",2182699.00,5986.46,1333.26,354.80,1033.51,3264.89,-
6,"June, 2021",2109221.00,5910.84,1913.76,362.32,1002.07,2632.69,-
7,"July, 2021",2379739.00,6109.60,1816.48,367.51,1044.83,2880.79,-
8,"August, 2021",2389790.00,6109.60,1633.30,349.52,1095.71,2937.83,-
9,"September, 2021",2389622.00,5957.41,2053.87,332.76,993.89,2576.89,-


In [299]:
# slice the dataframe
df = df.iloc[1:,:]

In [300]:
# drop the unrelevant columns from the dataframe
to_delete = df.columns[3:]
df = df.drop(labels=to_delete,axis=1)

In [301]:
df.head(30)

,Period,Oil Production (bbl),gas production (mmscf)
1,"January, 2021",2255226.00,5728.94
2,"February, 2021",1975758.00,5187.61
3,"March, 2021",2102802.00,5502.36
4,"April, 2021",2155621.00,6044.63
5,"May, 2021",2182699.00,5986.46
6,"June, 2021",2109221.00,5910.84
7,"July, 2021",2379739.00,6109.60
8,"August, 2021",2389790.00,6109.60
9,"September, 2021",2389622.00,5957.41
10,"October, 2021",2488514.00,6071.98


In [228]:
df.reset_index(drop=True,inplace=True)

In [229]:
df.head(20)

,OCTP / Sankofa-Gye Nyame Production for the Year 2020,Unnamed: 1,Unnamed: 2
0,"January, 2021",1500247.40,4190.78
1,"February, 2021",1315370.69,3781.97
2,"March, 2021",1399777.61,4414.32
3,"April, 2021",1382375.87,4220.77
4,"May, 2021",1402449.08,4522.95
5,"June, 2021",1302046.12,3873.82
6,"August, 2021",1314393.09,4813.50
7,"September, 2021",1214873.30,4094.45
8,"October, 2021",1264933.55,4868.60
9,"November, 2021",1187800.82,4732.39


In [209]:
df.head(15)

,Period,Oil Production (bbl),Associated Gas Production (MMscf),Non - Asssociated Gas Production (MMscf),Gas Re-injected (MMscf),Fuel Gas (MMscf),Flared Gas(MMscf),Sales Gas (MMscf)
1,"January, 2021",1500247.40,4190.78,6906.20,4683.72,357.95,64.11,5905.01
2,"February, 2021",1315370.69,3781.97,6473.50,4155.40,312.95,124.77,5572.65
3,"March, 2021",1399777.61,4414.32,5932.10,4404.13,343.72,28.97,5484.34
4,"April, 2021",1382375.87,4220.77,6276.20,3980.97,329.59,113.91,5984.50
5,"May, 2021",1402449.08,4522.95,5686.42,4324.06,343.01,30.72,5427.81
6,"June, 2021",1302046.12,3873.82,5438.06,3631.08,324.01,106.95,5168.62
7,"August, 2021",1314393.09,4813.50,5177.85,4604.96,360.41,46.53,4898.29
8,"September, 2021",1214873.30,4094.45,4656.10,3852.14,333.39,73.73,4401.50
9,"October, 2021",1264933.55,4868.60,5899.09,4680.14,354.30,22.64,5630.45
10,"November, 2021",1187800.82,4732.39,5586.05,4533.70,345.37,30.02,5330.79


In [248]:
# create the additional columns
df.columns = df.columns.str.lower()

In [249]:
df.head()

,period,oil production (bbl),associated gas production (mmscf)
0,JAN2020,1500247.40,4190.78
1,FEB2020,1315370.69,3781.97
2,MAR2020,1399777.61,4414.32
3,APR2020,1382375.87,4220.77
4,MAY2020,1402449.08,4522.95


In [291]:
#add the field column 
df['field'] = field_name.strip()

In [292]:
df.head()

,period,oil production (bbl),associated gas production (mmscf),field
1,JAN2021,1500247.40,4190.78,(OCTP) Sankofa/Gye Nyame
2,FEB2021,1315370.69,3781.97,(OCTP) Sankofa/Gye Nyame
3,MAR2021,1399777.61,4414.32,(OCTP) Sankofa/Gye Nyame
4,APR2021,1382375.87,4220.77,(OCTP) Sankofa/Gye Nyame
5,MAY2021,1402449.08,4522.95,(OCTP) Sankofa/Gye Nyame


In [293]:
df = pd.melt(df,id_vars=df['field'])

KeyError: "The following 'id_vars' are not present in the DataFrame: ['(OCTP) Sankofa/Gye Nyame']"

In [1]:
cd ..

C:\Users\OLOYEDE_O\PycharmProjects\scraper


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
import sys

root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)

In [4]:
# from scraper.jobs.ng_gov_dpr.nigerian_oil_supply import NigerianOilSupplyJob

from iea_scraper.core import factory

# job = NigerianOilSupplyJob()
# job = factory.get_scraper_job('ng_gov_dpr', 'nigerian_oil_supply')

# job = NigerianOilSupplyJob(full_load=True)
job = factory.get_scraper_job('gh_gov_petrocom', 'ghana_oil_supply', full_load=False)

2022-01-17 15:25:38,475 - iea_scraper.core.factory - DEBUG - Loading module iea_scraper.jobs.gh_gov_petrocom.ghana_oil_supply
2022-01-17 15:25:38,539 - iea_scraper.core.factory - DEBUG - Getting class GhanaOilSupplyJob
2022-01-17 15:25:39,080 - selenium.webdriver.remote.remote_connection - DEBUG - POST http://127.0.0.1:61914/session {"capabilities": {"firstMatch": [{}], "alwaysMatch": {"browserName": "chrome", "platformName": "any", "goog:chromeOptions": {"prefs": {"download.default_directory": "C:\\Users\\OLOYEDE_O\\PycharmProjects\\scraper\\filestore"}, "extensions": [], "args": ["--headless", "--disable-dev-shm-usage", "window-size=1920x1480"]}}}, "desiredCapabilities": {"browserName": "chrome", "version": "", "platform": "ANY", "goog:chromeOptions": {"prefs": {"download.default_directory": "C:\\Users\\OLOYEDE_O\\PycharmProjects\\scraper\\filestore"}, "extensions": [], "args": ["--headless", "--disable-dev-shm-usage", "window-size=1920x1480"]}}}
2022-01-17 15:25:39,082 - urllib3.con

In [ ]:
job.run()

2022-01-17 15:25:54,837 - iea_scraper.jobs.gh_gov_petrocom.ghana_oil_supply - INFO - getting the sources to the tables
2022-01-17 15:25:54,838 - iea_scraper.jobs.gh_gov_petrocom.ghana_oil_supply - DEBUG - parsing the main page to get table links at : https://www.petrocom.gov.gh/production-figures
2022-01-17 15:25:54,839 - selenium.webdriver.remote.remote_connection - DEBUG - POST http://127.0.0.1:61914/session/5a424176a55f9de8e4309d1a8bf13dbe/url {"url": "https://www.petrocom.gov.gh/production-figures"}
2022-01-17 15:25:59,168 - urllib3.connectionpool - DEBUG - http://127.0.0.1:61914 "POST /session/5a424176a55f9de8e4309d1a8bf13dbe/url HTTP/1.1" 200 14
2022-01-17 15:25:59,169 - selenium.webdriver.remote.remote_connection - DEBUG - Finished Request
2022-01-17 15:25:59,170 - iea_scraper.jobs.gh_gov_petrocom.ghana_oil_supply - INFO - successfully retrieved data from : https://www.petrocom.gov.gh/production-figures
2022-01-17 15:25:59,170 - selenium.webdriver.remote.remote_connection - DEBU

In [5]:
job.get_sources()

2022-01-17 15:20:29,217 - iea_scraper.jobs.gh_gov_petrocom.ghana_oil_supply - INFO - getting the sources to the tables
2022-01-17 15:20:29,218 - iea_scraper.jobs.gh_gov_petrocom.ghana_oil_supply - DEBUG - parsing the main page to get table links at : https://www.petrocom.gov.gh/production-figures
2022-01-17 15:20:29,218 - selenium.webdriver.remote.remote_connection - DEBUG - POST http://127.0.0.1:61671/session/223723510c7710c06549ba560aaef1bb/url {"url": "https://www.petrocom.gov.gh/production-figures"}
2022-01-17 15:20:34,253 - urllib3.connectionpool - DEBUG - http://127.0.0.1:61671 "POST /session/223723510c7710c06549ba560aaef1bb/url HTTP/1.1" 200 14
2022-01-17 15:20:34,255 - selenium.webdriver.remote.remote_connection - DEBUG - Finished Request
2022-01-17 15:20:34,256 - iea_scraper.jobs.gh_gov_petrocom.ghana_oil_supply - INFO - successfully retrieved data from : https://www.petrocom.gov.gh/production-figures
2022-01-17 15:20:34,257 - selenium.webdriver.remote.remote_connection - DEBU

In [6]:
[vars(s) for s in job.sources]

[{'code': 'GH_ten_2021',
  'url': 'https://www.petrocom.gov.gh/production-figures',
  'path': 'GH_ten_2021.html',
  'long_name': 'Ghana Monthly Oil Production for 2021-TEN',
  'meta_data': {'table_id': 'tablepress-11'}},
 {'code': 'GH_jubilee_2021',
  'url': 'https://www.petrocom.gov.gh/production-figures',
  'path': 'GH_jubilee_2021.html',
  'long_name': 'Ghana Monthly Oil Production for 2021-Jubilee',
  'meta_data': {'table_id': 'tablepress-10'}},
 {'code': 'GH_octp_sankofa-gyenyame_2021',
  'url': 'https://www.petrocom.gov.gh/production-figures',
  'path': 'GH_octp_sankofa-gyenyame_2021.html',
  'long_name': 'Ghana Monthly Oil Production for 2021-OCTP_Sankofa-GyeNyame',
  'meta_data': {'table_id': 'tablepress-12'}}]

In [7]:
job.download_and_get_checksum()

2022-01-17 15:20:40,322 - iea_scraper.core.job - DEBUG - download: True, parallel download: True
2022-01-17 15:20:40,323 - iea_scraper.core.utils - INFO - Executing function download_source() over 3 items with a maximum of 15 parallel workers.
2022-01-17 15:20:40,324 - iea_scraper.jobs.gh_gov_petrocom.ghana_oil_supply - INFO - Downloading source https://www.petrocom.gov.gh/production-figures to GH_ten_2021.html
2022-01-17 15:20:40,328 - selenium.webdriver.remote.remote_connection - DEBUG - POST http://127.0.0.1:61671/session/223723510c7710c06549ba560aaef1bb/element {"using": "css selector", "value": "[id=\"tablepress-11\"]"}
2022-01-17 15:20:40,330 - iea_scraper.jobs.gh_gov_petrocom.ghana_oil_supply - INFO - Downloading source https://www.petrocom.gov.gh/production-figures to GH_jubilee_2021.html
2022-01-17 15:20:40,333 - iea_scraper.jobs.gh_gov_petrocom.ghana_oil_supply - INFO - Downloading source https://www.petrocom.gov.gh/production-figures to GH_octp_sankofa-gyenyame_2021.html
202

In [8]:
job.transform()

2022-01-17 15:20:43,835 - selenium.webdriver.remote.remote_connection - DEBUG - GET http://127.0.0.1:61671/session/223723510c7710c06549ba560aaef1bb/source {}
2022-01-17 15:20:43,846 - urllib3.connectionpool - DEBUG - http://127.0.0.1:61671 "GET /session/223723510c7710c06549ba560aaef1bb/source HTTP/1.1" 200 100561
2022-01-17 15:20:43,848 - selenium.webdriver.remote.remote_connection - DEBUG - Finished Request
2022-01-17 15:20:43,875 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.
2022-01-17 15:20:43,884 - iea_scraper.jobs.gh_gov_petrocom.ghana_oil_supply - DEBUG - the temporary column has been dropped
2022-01-17 15:20:43,893 - selenium.webdriver.remote.remote_connection - DEBUG - GET http://127.0.0.1:61671/session/223723510c7710c06549ba560aaef1bb/source {}
2022-01-17 15:20:43,903 - urllib3.connectionpool - DEBUG - http://127.0.0.1:61671 "GET /session/223723510c7710c06549ba560aaef1bb/source HTTP/1.1" 200 100561
2022-01-17 15:20:43,906 - selenium.webdriver.remote.remote_connecti

In [17]:
job.data[job.data['entity']=='GH_GOV_PETROCOM_JUBILEE']

,entity,period,product,value,unit,provider,frequency,area,flow,original,source
0,GH_GOV_PETROCOM_JUBILEE,JAN2021,CRUDEOIL,72.749,KBD,GH_GOV_PETROCOM,Monthly,GHANA,SUPPLY,True,GH_jubilee_2021
1,GH_GOV_PETROCOM_JUBILEE,FEB2021,CRUDEOIL,70.563,KBD,GH_GOV_PETROCOM,Monthly,GHANA,SUPPLY,True,GH_jubilee_2021
2,GH_GOV_PETROCOM_JUBILEE,MAR2021,CRUDEOIL,67.832,KBD,GH_GOV_PETROCOM,Monthly,GHANA,SUPPLY,True,GH_jubilee_2021
3,GH_GOV_PETROCOM_JUBILEE,APR2021,CRUDEOIL,71.854,KBD,GH_GOV_PETROCOM,Monthly,GHANA,SUPPLY,True,GH_jubilee_2021
4,GH_GOV_PETROCOM_JUBILEE,MAY2021,CRUDEOIL,70.41,KBD,GH_GOV_PETROCOM,Monthly,GHANA,SUPPLY,True,GH_jubilee_2021
5,GH_GOV_PETROCOM_JUBILEE,JUN2021,CRUDEOIL,70.307,KBD,GH_GOV_PETROCOM,Monthly,GHANA,SUPPLY,True,GH_jubilee_2021
6,GH_GOV_PETROCOM_JUBILEE,JUL2021,CRUDEOIL,76.766,KBD,GH_GOV_PETROCOM,Monthly,GHANA,SUPPLY,True,GH_jubilee_2021
7,GH_GOV_PETROCOM_JUBILEE,AUG2021,CRUDEOIL,77.09,KBD,GH_GOV_PETROCOM,Monthly,GHANA,SUPPLY,True,GH_jubilee_2021
8,GH_GOV_PETROCOM_JUBILEE,SEP2021,CRUDEOIL,79.654,KBD,GH_GOV_PETROCOM,Monthly,GHANA,SUPPLY,True,GH_jubilee_2021
9,GH_GOV_PETROCOM_JUBILEE,OCT2021,CRUDEOIL,80.275,KBD,GH_GOV_PETROCOM,Monthly,GHANA,SUPPLY,True,GH_jubilee_2021


In [10]:
job.run()

2021-11-24 12:14:27,282 - scraper.jobs.gh_gov_petrocom.ghana_oil_supply - INFO - getting the sources to the tables
2021-11-24 12:14:27,282 - scraper.jobs.gh_gov_petrocom.ghana_oil_supply - DEBUG - parsing the main page to get table links at : https://www.petrocom.gov.gh/production-figures
2021-11-24 12:14:27,283 - selenium.webdriver.remote.remote_connection - DEBUG - POST http://127.0.0.1:62588/session/dea6d8a413086fc72a1baea4529b99b9/url {"url": "https://www.petrocom.gov.gh/production-figures"}
2021-11-24 12:14:27,406 - urllib3.connectionpool - DEBUG - http://127.0.0.1:62588 "POST /session/dea6d8a413086fc72a1baea4529b99b9/url HTTP/1.1" 200 14
2021-11-24 12:14:27,407 - selenium.webdriver.remote.remote_connection - DEBUG - Finished Request
2021-11-24 12:14:27,407 - scraper.jobs.gh_gov_petrocom.ghana_oil_supply - INFO - successfully retrieved data from : https://www.petrocom.gov.gh/production-figures
2021-11-24 12:14:27,409 - selenium.webdriver.remote.remote_connection - DEBUG - POST htt

OSError: Issue for loop 2: 
 b'{"message": "Internal Server Error"}\n'

In [38]:
job.dynamic_dim['provider']

[]

In [9]:
job.data[50:575]

,entity,period,product,value,unit,provider,frequency,area,flow,original,source
8,OCTP_Sankofa_GyeNyame,NOV2021,CRUDEOIL,33.654,KBD,GH_GOV_PETROCOM,Monthly,GHANA,SUPPLY,True,GH_octp_sankofa-gyenyame_2021
9,OCTP_Sankofa_GyeNyame,JAN2021,NATGAS,4190.78,MMSCF,GH_GOV_PETROCOM,Monthly,GHANA,SUPPLY,True,GH_octp_sankofa-gyenyame_2021
10,OCTP_Sankofa_GyeNyame,FEB2021,NATGAS,3781.97,MMSCF,GH_GOV_PETROCOM,Monthly,GHANA,SUPPLY,True,GH_octp_sankofa-gyenyame_2021
11,OCTP_Sankofa_GyeNyame,MAR2021,NATGAS,4414.32,MMSCF,GH_GOV_PETROCOM,Monthly,GHANA,SUPPLY,True,GH_octp_sankofa-gyenyame_2021
12,OCTP_Sankofa_GyeNyame,APR2021,NATGAS,4220.77,MMSCF,GH_GOV_PETROCOM,Monthly,GHANA,SUPPLY,True,GH_octp_sankofa-gyenyame_2021
13,OCTP_Sankofa_GyeNyame,MAY2021,NATGAS,4522.95,MMSCF,GH_GOV_PETROCOM,Monthly,GHANA,SUPPLY,True,GH_octp_sankofa-gyenyame_2021
14,OCTP_Sankofa_GyeNyame,JUN2021,NATGAS,3873.82,MMSCF,GH_GOV_PETROCOM,Monthly,GHANA,SUPPLY,True,GH_octp_sankofa-gyenyame_2021
15,OCTP_Sankofa_GyeNyame,AUG2021,NATGAS,4813.50,MMSCF,GH_GOV_PETROCOM,Monthly,GHANA,SUPPLY,True,GH_octp_sankofa-gyenyame_2021
16,OCTP_Sankofa_GyeNyame,SEP2021,NATGAS,4094.45,MMSCF,GH_GOV_PETROCOM,Monthly,GHANA,SUPPLY,True,GH_octp_sankofa-gyenyame_2021
17,OCTP_Sankofa_GyeNyame,NOV2021,NATGAS,3120.31,MMSCF,GH_GOV_PETROCOM,Monthly,GHANA,SUPPLY,True,GH_octp_sankofa-gyenyame_2021


In [397]:
job.transheadrm_entity(job.data['entity'])

2021-10-01 12:03:04,841 - scraper.core.job - DEBUG - remove_existing_dynamic_dim: query - http://vipenta.iea.org:8000/dimension/entity
2021-10-01 12:03:04,841 - scraper.core.job - DEBUG - remove_existing_dynamic_dim: query - http://vipenta.iea.org:8000/dimension/entity
2021-10-01 12:03:04,841 - scraper.core.job - DEBUG - remove_existing_dynamic_dim: query - http://vipenta.iea.org:8000/dimension/entity
2021-10-01 12:03:04,848 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): vipenta.iea.org:8000
2021-10-01 12:03:04,848 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): vipenta.iea.org:8000
2021-10-01 12:03:04,848 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): vipenta.iea.org:8000
2021-10-01 12:03:21,204 - urllib3.connectionpool - DEBUG - http://vipenta.iea.org:8000 "GET /dimension/entity HTTP/1.1" 200 17239958
2021-10-01 12:03:21,204 - urllib3.connectionpool - DEBUG - http://vipenta.iea.org:8000 "GET /dimension/entity HTTP/1.1" 

0                       GH_GOV_PETROCOM_TEN
1                       GH_GOV_PETROCOM_TEN
2                       GH_GOV_PETROCOM_TEN
3                       GH_GOV_PETROCOM_TEN
4                       GH_GOV_PETROCOM_TEN
5                       GH_GOV_PETROCOM_TEN
6                       GH_GOV_PETROCOM_TEN
7                       GH_GOV_PETROCOM_TEN
8                       GH_GOV_PETROCOM_TEN
9                       GH_GOV_PETROCOM_TEN
10                      GH_GOV_PETROCOM_TEN
11                      GH_GOV_PETROCOM_TEN
12                      GH_GOV_PETROCOM_TEN
13                      GH_GOV_PETROCOM_TEN
0                   GH_GOV_PETROCOM_JUBILEE
1                   GH_GOV_PETROCOM_JUBILEE
2                   GH_GOV_PETROCOM_JUBILEE
3                   GH_GOV_PETROCOM_JUBILEE
4                   GH_GOV_PETROCOM_JUBILEE
5                   GH_GOV_PETROCOM_JUBILEE
6                   GH_GOV_PETROCOM_JUBILEE
7                   GH_GOV_PETROCOM_JUBILEE
8                   GH_GOV_PETRO

In [395]:
job.data['entity'] + 'GH_GOV_PETROCOM'

0                       TENGH_GOV_PETROCOM
1                       TENGH_GOV_PETROCOM
2                       TENGH_GOV_PETROCOM
3                       TENGH_GOV_PETROCOM
4                       TENGH_GOV_PETROCOM
5                       TENGH_GOV_PETROCOM
6                       TENGH_GOV_PETROCOM
7                       TENGH_GOV_PETROCOM
8                       TENGH_GOV_PETROCOM
9                       TENGH_GOV_PETROCOM
10                      TENGH_GOV_PETROCOM
11                      TENGH_GOV_PETROCOM
12                      TENGH_GOV_PETROCOM
13                      TENGH_GOV_PETROCOM
0                   JubileeGH_GOV_PETROCOM
1                   JubileeGH_GOV_PETROCOM
2                   JubileeGH_GOV_PETROCOM
3                   JubileeGH_GOV_PETROCOM
4                   JubileeGH_GOV_PETROCOM
5                   JubileeGH_GOV_PETROCOM
6                   JubileeGH_GOV_PETROCOM
7                   JubileeGH_GOV_PETROCOM
8                   JubileeGH_GOV_PETROCOM
9          

In [401]:
job.dynamic_dim['entity']

[{'code': 'GH_GOV_PETROCOM_TEN', 'long_name': 'TEN', 'category': 'field'},
 {'code': 'GH_GOV_PETROCOM_JUBILEE',
  'long_name': 'JUBILEE',
  'category': 'field'},
 {'code': 'GH_GOV_PETROCOM_OCTP_SANKOFA_GYENYAME',
  'long_name': 'OCTP_SANKOFA_GYENYAME',
  'category': 'field'}]

In [15]:
job.data['entity'].unique()

array(['GH_GOV_PETROCOM_TEN', 'GH_GOV_PETROCOM_JUBILEE',
       'GH_GOV_PETROCOM_OCTP_SANKOFA_GYENYAME'], dtype=object)

In [338]:
import pandas as pd
sub_pd = job.data[['period','value','field','gas production (mmscf)']]

In [355]:
pd.melt(sub_pd,id_vars=['field','period'],value_vars=['value','gas production (mmscf)'],var_name='product')

,field,period,product,value
0,GH_GOV_PETROCOM_TEN,JAN2021,value,39.575
1,GH_GOV_PETROCOM_TEN,FEB2021,value,38.219
2,GH_GOV_PETROCOM_TEN,MAR2021,value,38.68
3,GH_GOV_PETROCOM_TEN,APR2021,value,36.464
4,GH_GOV_PETROCOM_TEN,MAY2021,value,36.181
5,GH_GOV_PETROCOM_TEN,JUN2021,value,32.423
6,GH_GOV_PETROCOM_TEN,JUL2021,value,31.2
7,GH_GOV_PETROCOM_Jubilee,JAN2021,value,72.749
8,GH_GOV_PETROCOM_Jubilee,FEB2021,value,70.563
9,GH_GOV_PETROCOM_Jubilee,MAR2021,value,67.832


In [217]:
import pandas as pd
pd.to_datetime(job.data['period'])

1   2021-01-01
2   2021-02-01
3   2021-03-01
4   2021-04-01
5   2021-05-01
6   2021-06-01
7   2021-07-01
1   2021-01-01
2   2021-02-01
3   2021-03-01
4   2021-04-01
5   2021-05-01
6   2021-06-01
7   2021-07-01
1   2021-01-01
2   2021-02-01
3   2021-03-01
4   2021-04-01
5   2021-05-01
6   2021-06-01
7   2021-07-01
Name: period, dtype: datetime64[ns]

In [3]:
#%cd ..
import requests
from scraper.settings import REQUESTS_HEADERS, SSL_CERTIFICATE_PATH

r = requests.get('https://www.nuprc.gov.ng/index.php', headers=REQUESTS_HEADERS, verify=SSL_CERTIFICATE_PATH)

r.raise_for_status()

HTTPError: 503 Server Error: Service Temporarily Unavailable for url: https://www.nuprc.gov.ng/index.php